# Federated Never Ending Learning

## Alignment Layer

In [152]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [153]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [154]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

In [155]:
df = pd.read_csv('datasets/CICIDS_test.csv', skipinitialspace=True)

In [156]:
from sklearn.preprocessing import MinMaxScaler, Normalizer
# from keras.utils import to_categorical

# # removing any class with less than 'thresh' samples
# thresh = 1000
# counts = df[['Label']].value_counts().to_dict()
# keep = [x[0] for x in counts if counts[x] >= thresh]
# df = df.loc[df['Label'].isin(keep)]


X = df.loc[:, df.columns != 'Label']
X = Normalizer().fit_transform(X)
X = MinMaxScaler().fit_transform(X)
Y = df[['Label']].to_numpy()

In [157]:
print(X.shape)
print(Y.shape)
print(len(np.unique(Y)))
pd.value_counts(Y.ravel())

(566149, 78)
(566149, 1)
15


0     454620
4      46215
10     31786
2      25606
3       2059
7       1588
11      1179
6       1159
5       1100
1        393
12       301
14       130
9          7
13         4
8          2
dtype: int64

## Classification Models

In [158]:
class BinaryClassification:
    def __init__(self, input_dims, layers):
        self.model = Sequential(name="server_model")
        self.model.add(Dense(
            layers[0],
            input_shape=(input_dims,),
            activation='relu'
        ))
        for l in layers[1:]:
            self.model.add(Dense(l, activation='relu'))
        self.model.add(Dense(1, activation='sigmoid'))
        self.model.compile(
            loss='binary_crossentropy',
            optimizer='adam',
            metrics=['accuracy', ]
        )
        self.model.summary()
    
    def fit(self, X, Y, epochs, batch_size):
        Y_bin = np.asarray([
            0 if y == 0 else 1 for y in Y.ravel()
        ]).reshape(-1, 1)
        # print(Y_bin.shape)
        # print(pd.value_counts(Y_bin.ravel()))
        self.model.fit(X, Y_bin, epochs=epochs, batch_size=batch_size)

    def predict(self, X, Y):
        Y_bin = np.asarray([
            0 if y == 0 else 1 for y in Y.ravel()
        ]).reshape(-1, 1)
        # print(pd.value_counts(Y_bin.ravel()))
        
        preds = self.model.predict(X)
        preds = np.where(preds > 0.5, 1, 0)

        # print(Y_bin.shape)
        # print(preds.shape)
        print(classification_report(Y_bin, preds))
        print(confusion_matrix(Y_bin, preds))


In [159]:
class MulticlassClassification:
    def __init__(self, input_dims, layers, num_classes):
        self.model = Sequential(name="server_model")
        self.model.add(Dense(
            layers[0],
            input_shape=(input_dims,),
            activation='relu'
        ))
        for l in layers[1:]:
            self.model.add(Dense(l, activation='relu'))
        self.model.add(Dense(num_classes, activation='softmax'))
        self.model.compile(
            loss='sparse_categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy']
        )
        self.model.summary()
    
    def fit(self, X, Y, epochs, batch_size):
        # print(pd.value_counts(Y.ravel()))
        self.model.fit(X, Y.ravel(), epochs=epochs, batch_size=batch_size)

    def predict(self, X, Y):
        # print(pd.value_counts(Y.ravel()))
        
        preds = self.model.predict(X)
        preds = [np.argmax(p) for p in preds]
        
        print(classification_report(Y, preds))
        print(confusion_matrix(Y, preds))


## Non-Federated

In [160]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.7, random_state=1)

print(X_train.shape)
print(Y_train.shape)
print(np.unique(Y_train))
print(pd.value_counts(Y_train.ravel()))

print(X_test.shape)
print(Y_test.shape)
print(np.unique(Y_test))
print(pd.value_counts(Y_test.ravel()))

(396304, 78)
(396304, 1)
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
0     318395
4      32391
10     22137
2      17891
3       1417
7       1123
11       827
6        786
5        780
1        257
12       197
14        92
9          7
13         2
8          2
dtype: int64
(169845, 78)
(169845, 1)
[ 0  1  2  3  4  5  6  7 10 11 12 13 14]
0     136225
4      13824
10      9649
2       7715
3        642
7        465
6        373
11       352
5        320
1        136
12       104
14        38
13         2
dtype: int64


### Binary Classification into Benign/Malicious

In [161]:
model_binary = BinaryClassification(
    input_dims=78,
    layers=[20, 20, 20]
)
model_binary.fit(X_train, Y_train, epochs=20, batch_size=32)

Model: "server_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_172 (Dense)           (None, 20)                1580      
                                                                 
 dense_173 (Dense)           (None, 20)                420       
                                                                 
 dense_174 (Dense)           (None, 20)                420       
                                                                 
 dense_175 (Dense)           (None, 1)                 21        
                                                                 
Total params: 2,441
Trainable params: 2,441
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
12385/12385 [==============================] - 42s 3ms/step - loss: 0.0984 - accuracy: 0.9641
Epoch 2/20
12385/12385 [==============================] - 39s 3ms/step - loss: 

In [162]:
model_binary.predict(X_test, Y_test)

5308/5308 [==============================] - 11s 2ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    136225
           1       0.99      0.98      0.99     33620

    accuracy                           0.99    169845
   macro avg       0.99      0.99      0.99    169845
weighted avg       0.99      0.99      0.99    169845

[[135821    404]
 [   530  33090]]


### SoftMax A

In [163]:
model_sf = MulticlassClassification(
    input_dims=78,
    layers=[20, 20, 20],
    num_classes=15
)
model_sf.fit(X_train, Y_train, epochs=20, batch_size=32)

Model: "server_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_176 (Dense)           (None, 20)                1580      
                                                                 
 dense_177 (Dense)           (None, 20)                420       
                                                                 
 dense_178 (Dense)           (None, 20)                420       
                                                                 
 dense_179 (Dense)           (None, 15)                315       
                                                                 
Total params: 2,735
Trainable params: 2,735
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
12385/12385 [==============================] - 34s 3ms/step - loss: 0.1852 - accuracy: 0.9440
Epoch 2/20
12385/12385 [==============================] - 34s 3ms/step - loss: 

In [164]:
model_sf.predict(X_test, Y_test)

5308/5308 [==============================] - 10s 2ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    136225
           1       0.59      0.60      0.60       136
           2       0.98      0.99      0.98      7715
           3       0.93      0.97      0.95       642
           4       0.98      0.99      0.99     13824
           5       0.80      0.71      0.75       320
           6       0.88      0.69      0.77       373
           7       1.00      0.98      0.99       465
          10       0.99      1.00      0.99      9649
          11       0.97      0.95      0.96       352
          12       0.00      0.00      0.00       104
          13       0.00      0.00      0.00         2
          14       0.00      0.00      0.00        38

    accuracy                           0.99    169845
   macro avg       0.70      0.68      0.69    169845
weighted avg       0.99      0.99      0.99    169845

[[135742     56     6

## Anomaly Detection Using Isolation Forest

## Federated Learning

In [165]:
X_1, X_2, Y_1, Y_2 = train_test_split(X_train, Y_train, train_size=0.5, random_state=1)

In [166]:
print(X_1.shape)
print(Y_1.shape)
print(np.unique(Y_1))
print(pd.value_counts(Y_1.ravel()))
print(X_2.shape)
print(Y_2.shape)
print(np.unique(Y_2))
print(pd.value_counts(Y_2.ravel()))

(198152, 78)
(198152, 1)
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
0     158990
4      16294
10     11120
2       8918
3        735
7        561
11       440
5        409
6        392
1        139
12        99
14        51
9          2
8          1
13         1
dtype: int64
(198152, 78)
(198152, 1)
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
0     159405
4      16097
10     11017
2       8973
3        682
7        562
6        394
11       387
5        371
1        118
12        98
14        41
9          5
8          1
13         1
dtype: int64


### Binary Classification

In [167]:
model_1 = BinaryClassification(78, [20, 20, 20])
model_1.fit(X_1, Y_1, epochs=20, batch_size=32)

Model: "server_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_180 (Dense)           (None, 20)                1580      
                                                                 
 dense_181 (Dense)           (None, 20)                420       
                                                                 
 dense_182 (Dense)           (None, 20)                420       
                                                                 
 dense_183 (Dense)           (None, 1)                 21        
                                                                 
Total params: 2,441
Trainable params: 2,441
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
6193/6193 [==============================] - 19s 3ms/step - loss: 0.1282 - accuracy: 0.9508
Epoch 2/20
6193/6193 [==============================] - 18s 3ms/step - loss: 0.06

In [168]:
model_1.predict(X_test, Y_test)

5308/5308 [==============================] - 10s 2ms/step
              precision    recall  f1-score   support

           0       1.00      0.99      1.00    136225
           1       0.98      0.98      0.98     33620

    accuracy                           0.99    169845
   macro avg       0.99      0.99      0.99    169845
weighted avg       0.99      0.99      0.99    169845

[[135541    684]
 [   635  32985]]


In [169]:
model_2 = BinaryClassification(78, [20, 20, 20])
model_2.fit(X_2, Y_2, epochs=20, batch_size=32)

Model: "server_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_184 (Dense)           (None, 20)                1580      
                                                                 
 dense_185 (Dense)           (None, 20)                420       
                                                                 
 dense_186 (Dense)           (None, 20)                420       
                                                                 
 dense_187 (Dense)           (None, 1)                 21        
                                                                 
Total params: 2,441
Trainable params: 2,441
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
6193/6193 [==============================] - 18s 3ms/step - loss: 0.1362 - accuracy: 0.9451
Epoch 2/20
6193/6193 [==============================] - 18s 3ms/step - loss: 0.07

In [170]:
model_2.predict(X_test, Y_test)

5308/5308 [==============================] - 10s 2ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    136225
           1       0.99      0.98      0.98     33620

    accuracy                           0.99    169845
   macro avg       0.99      0.99      0.99    169845
weighted avg       0.99      0.99      0.99    169845

[[135830    395]
 [   614  33006]]


In [171]:
model_3 = BinaryClassification(78, [20, 20, 20])
model_3.model.set_weights(np.mean(
        np.array([
            model_1.model.get_weights(),
            model_2.model.get_weights()
        ]),
        axis=0
    )
)

Model: "server_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_188 (Dense)           (None, 20)                1580      
                                                                 
 dense_189 (Dense)           (None, 20)                420       
                                                                 
 dense_190 (Dense)           (None, 20)                420       
                                                                 
 dense_191 (Dense)           (None, 1)                 21        
                                                                 
Total params: 2,441
Trainable params: 2,441
Non-trainable params: 0
_________________________________________________________________


In [172]:
model_3.predict(X_test, Y_test)

5308/5308 [==============================] - 10s 2ms/step
              precision    recall  f1-score   support

           0       0.84      0.94      0.89    136225
           1       0.52      0.25      0.33     33620

    accuracy                           0.81    169845
   macro avg       0.68      0.60      0.61    169845
weighted avg       0.77      0.81      0.78    169845

[[128564   7661]
 [ 25323   8297]]


### Multiclass Classification

In [173]:
model_sf_1 = MulticlassClassification(
    78,
    [20, 20, 20],
    num_classes=15
)
model_sf_1.fit(X_1, Y_1, epochs=20, batch_size=32)

Model: "server_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_192 (Dense)           (None, 20)                1580      
                                                                 
 dense_193 (Dense)           (None, 20)                420       
                                                                 
 dense_194 (Dense)           (None, 20)                420       
                                                                 
 dense_195 (Dense)           (None, 15)                315       
                                                                 
Total params: 2,735
Trainable params: 2,735
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
6193/6193 [==============================] - 18s 3ms/step - loss: 0.2531 - accuracy: 0.9260
Epoch 2/20
6193/6193 [==============================] - 17s 3ms/step - loss: 0.10

In [174]:
model_sf_1.predict(X_test, Y_test)

5308/5308 [==============================] - 12s 2ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    136225
           1       0.54      0.60      0.56       136
           2       0.98      0.99      0.98      7715
           3       0.94      0.93      0.94       642
           4       0.99      0.99      0.99     13824
           5       0.79      0.66      0.72       320
           6       0.86      0.70      0.77       373
           7       0.99      0.98      0.98       465
          10       0.99      1.00      0.99      9649
          11       0.93      0.98      0.95       352
          12       0.00      0.00      0.00       104
          13       0.00      0.00      0.00         2
          14       0.00      0.00      0.00        38

    accuracy                           0.99    169845
   macro avg       0.69      0.68      0.68    169845
weighted avg       0.99      0.99      0.99    169845

[[135718     70     7

In [175]:
model_sf_2 = MulticlassClassification(
    78,
    [20, 20, 20],
    num_classes=15
)
model_sf_2.fit(X_2, Y_2, epochs=20, batch_size=32)

Model: "server_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_196 (Dense)           (None, 20)                1580      
                                                                 
 dense_197 (Dense)           (None, 20)                420       
                                                                 
 dense_198 (Dense)           (None, 20)                420       
                                                                 
 dense_199 (Dense)           (None, 15)                315       
                                                                 
Total params: 2,735
Trainable params: 2,735
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
6193/6193 [==============================] - 18s 3ms/step - loss: 0.2361 - accuracy: 0.9278
Epoch 2/20
6193/6193 [==============================] - 17s 3ms/step - loss: 0.10

In [176]:
model_sf_2.predict(X_test, Y_test)

5308/5308 [==============================] - 10s 2ms/step
              precision    recall  f1-score   support

           0       0.99      1.00      1.00    136225
           1       0.86      0.35      0.50       136
           2       0.98      0.99      0.98      7715
           3       0.95      0.93      0.94       642
           4       0.99      0.98      0.99     13824
           5       0.78      0.16      0.26       320
           6       0.95      0.65      0.77       373
           7       0.97      0.98      0.97       465
          10       0.99      1.00      0.99      9649
          11       0.95      0.96      0.95       352
          12       0.00      0.00      0.00       104
          13       0.00      0.00      0.00         2
          14       0.00      0.00      0.00        38

    accuracy                           0.99    169845
   macro avg       0.72      0.61      0.64    169845
weighted avg       0.99      0.99      0.99    169845

[[135893      8     6

In [177]:
model_sf_3 = MulticlassClassification(
    78,
    [20, 20, 20],
    num_classes=15
)
model_sf_3.model.set_weights(np.mean(
        np.array([
            model_sf_1.model.get_weights(),
            model_sf_2.model.get_weights()
        ]),
        axis=0
    )
)

Model: "server_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_200 (Dense)           (None, 20)                1580      
                                                                 
 dense_201 (Dense)           (None, 20)                420       
                                                                 
 dense_202 (Dense)           (None, 20)                420       
                                                                 
 dense_203 (Dense)           (None, 15)                315       
                                                                 
Total params: 2,735
Trainable params: 2,735
Non-trainable params: 0
_________________________________________________________________


In [178]:
model_sf_3.predict(X_test, Y_test)


5308/5308 [==============================] - 10s 2ms/step
              precision    recall  f1-score   support

           0       0.84      0.98      0.90    136225
           1       0.00      0.00      0.00       136
           2       0.86      0.13      0.22      7715
           3       0.00      0.00      0.00       642
           4       0.02      0.00      0.01     13824
           5       0.00      0.02      0.00       320
           6       0.00      0.00      0.00       373
           7       0.00      0.00      0.00       465
          10       0.00      0.00      0.00      9649
          11       0.00      0.00      0.00       352
          12       0.00      0.00      0.00       104
          13       0.00      0.00      0.00         2
          14       0.00      0.00      0.00        38

    accuracy                           0.79    169845
   macro avg       0.13      0.09      0.09    169845
weighted avg       0.71      0.79      0.73    169845

[[133102      0     1